## Python Function Wrappers (Decorators)

### 1. What is a Decorator?

A **decorator** is a higher-order function that takes a function as input and extends or modifies its behavior (including restriction).
It’s a powerful tool in Python used to add functionality in a modular and reusable way.

### 2. Basic Structure of a Decorator

A simple decorator that adds a logging feature to a function.

In [23]:
def log_function_call(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with args: {args}, kwargs: {kwargs}")
        result = func(*args, **kwargs)
        print(f"{func.__name__} returned: {result}")
        return result
    return wrapper

@log_function_call
def logged_function(a, b):
    return a + b

def add_function(a, b):
    return a + b

In [ ]:
add_function(1, 2)

In [ ]:
logged_function(1, 2)

### 3.1. Timing Execution

Wrappers can be used to time how long a function takes to execute. 

In [ ]:
import time

def time_execution(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Execution time of {func.__name__}: {end_time - start_time:.6f} seconds")
        return result
    return wrapper

@time_execution
def slow_function():
    time.sleep(2) 
    return "Function Finished!"

slow_function()

### 3.2. Access Control

Decorators can control access to specific functions, which is useful for checking permissions or authorizing access.

In [27]:
def requires_admin(func):
    def wrapper(*args, **kwargs):
        user_role = kwargs.get('role', 'guest') # Checks role from kwargs, sets default to 'guest' if not found
        if user_role != 'admin':
            print("Access denied: Admins only!")
            return
        return func(*args, **kwargs) # Calls the original function if the user is an admin
    return wrapper

@requires_admin
def delete_database(*args, **kwargs):
    print("Database deleted!")

In [ ]:
delete_database(role='guest')

In [ ]:
delete_database(role='admin')

Can read env variables is access levels are set at system level

In [30]:
def requires_admin(func):
    def wrapper(*args, **kwargs):
        user_role = env_var_role # Checks role from env, just rpetend its sys.argv....
        if user_role != 'admin':
            print("Access denied: Admins only!")
            return
        return func(*args, **kwargs) # Calls the original function if the user is an admin
    return wrapper

@requires_admin
def delete_database(*args, **kwargs):
    print("Database deleted!")

In [ ]:
env_var_role = 'guest'
delete_database()

In [ ]:
env_var_role = 'admin'
delete_database()

### 3.3. Caching Expensive Operations

If a function is expensive to compute but often called with the same arguments, a decorator can be used to cache results and return them for repeated inputs.

In [33]:
def cache_func(func):
    """Decorator to cache results of function calls."""
    cache = {}
    def wrapper(*args):
        if args in cache:
            print(f"Returning cached result for {args}")
            return cache[args]
        result = func(*args)
        cache[args] = result
        return result
    return wrapper

@cache_func
def fibonacci(n):
    if n in (0, 1):
        return n
    return fibonacci(n-1) + fibonacci(n-2)

@cache_func
def simple_add(a):
    return a + 10

In [ ]:
simple_add(10)

In [ ]:
simple_add(10)

In [ ]:
print(fibonacci(10))

In [ ]:
print(fibonacci(10)) # Result fetched from cache

### 3.4. Input Validation

You can use a function wrapper to validate inputs before the function logic is executed.

In [38]:
def validate_non_negative(func):
    """Decorator to ensure arguments are non-negative."""
    def wrapper(*args, **kwargs):
        if any(arg < 0 for arg in args):
            raise ValueError("Negative values are not allowed!")
        return func(*args, **kwargs)
    return wrapper

@validate_non_negative
def factorial(n):
    """Computes the factorial of a number."""
    if n == 0:
        return 1
    return n * factorial(n - 1)

In [ ]:
factorial(5)

In [ ]:
try:
    factorial(-1)
except ValueError as e:
    print(e)

### 3.5. Retrying on Failure

Wrappers can also implement retry logic for functions that may fail due to external issues (e.g., network calls).

In [41]:
def retry_on_failure(retries=3):
    """Decorator to retry a function if it fails."""
    def decorator(func):
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < retries:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"Attempt {attempts + 1} failed: {e}")
                    attempts += 1
            raise Exception(f"Function {func.__name__} failed after {retries} retries")
        return wrapper
    return decorator

In [43]:
# Simulate a function that fails intermittently
import random

@retry_on_failure(retries=1)
def unreliable_function():
    """Simulates a function that fails randomly."""
    if random.choice([True, False]):
        raise ValueError("Random failure!")
    return "Success!"

In [ ]:
try:
    print(unreliable_function())
except Exception as e:
    print(e)

### 4.1. Nested Decorators

You can apply multiple decorators to a single function. This is useful when you need to layer behavior.

In [49]:
def combined_decorator(func):
    """Combines logging, timing, and caching in one decorator."""
    @log_function_call
    @time_execution
    @cache_func
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@combined_decorator
def slow_fibonacci(n):
    if n in (0, 1):
        return n
    return slow_fibonacci(n-1) + slow_fibonacci(n-2)

In [ ]:
slow_fibonacci(4)

### 4.2. Class-based Decorators

Function decorators aren't limited to functions—they can be applied to classes as well.
Here’s how you can create a class-based decorator that wraps around a function, allowing for more complex state management.

In [54]:
class CountCalls:
    """Decorator that counts how many times a function is called."""
    def __init__(self, func):
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"{self.func.__name__} has been called {self.num_calls} times.")
        return self.func(*args, **kwargs)

# Apply the class-based decorator
@CountCalls
def greet(name):
    print(f"Hello, {name}!")

In [ ]:
greet("Alice")
greet("Bob")
greet("Charlie")

### 4.3. Parameterized Decorators

Sometimes you want your decorator to accept arguments (e.g., to customize behavior).
A decorator that accepts arguments is known as a **parameterized decorator**.


In [57]:
def repeat(num_times):
    """Decorator that repeats the function call a given number of times."""
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator

# Apply the parameterized decorator
@repeat(num_times=3)
def say_hello(name):
    print(f"Hello, {name}!")

In [ ]:
say_hello("Python Stream!")

In [65]:
## Real world example

def inactive_wrapper(blocked_response):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if ACTIVE:
                return func(*args, **kwargs)
            else:
                print("Command execution blocked.")
                return blocked_response

        return wrapper

    return decorator

@inactive_wrapper(blocked_response="Simulated Message Sent!")
def send_outbound_message(message):
    print(f"Sending message to API: {message}")
    return "Message sent!"

In [ ]:
ACTIVE = True
send_outbound_message("Charge a battery")

In [ ]:
ACTIVE = False
send_outbound_message("Charge a battery")

### 4.4. Stateful Decorators (Tracking State)

A decorator can also maintain internal state across multiple calls.
This is particularly useful when you need to track information over time, like rate-limiting calls or counting usage.


In [68]:
class RateLimiter:
    """Decorator to limit the rate at which a function can be called."""
    def __init__(self, max_calls, period):
        self.max_calls = max_calls
        self.period = period
        self.calls = []
    
    def __call__(self, func):
        def wrapper(*args, **kwargs):
            now = time.time()
            self.calls = [call for call in self.calls if call > now - self.period]
            if len(self.calls) >= self.max_calls:
                raise Exception(f"Rate limit exceeded: Max {self.max_calls} calls per {self.period} seconds")
            self.calls.append(now)
            return func(*args, **kwargs)
        return wrapper

@RateLimiter(max_calls=3, period=5)  # Max 3 calls every 5 seconds
def api_call():
    print("API call executed!")

In [ ]:
try:
    api_call()
    api_call()
    api_call()
    api_call()  # This one should raise an exception
except Exception as e:
    print(e)

### 4.5 Coroutines and Async Decorators

In Python's asynchronous programming, decorators are equally useful.
You can create decorators that work with `async` functions to provide functionality like retry logic or timing.

In [70]:
import asyncio

def async_retry_on_failure(retries=3):
    """Async decorator to retry a coroutine on failure."""
    def decorator(func):
        async def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < retries:
                try:
                    return await func(*args, **kwargs)
                except Exception as e:
                    print(f"Attempt {attempts + 1} failed: {e}")
                    attempts += 1
            raise Exception(f"Function {func.__name__} failed after {retries} retries")
        return wrapper
    return decorator

# Define an async function with the retry decorator

@async_retry_on_failure(retries=5)
async def unreliable_async_function():
    """Simulates an async function that fails randomly."""
    if random.choice([True, False]):
        raise ValueError("Random async failure!")
    return "Async Success!"

In [ ]:
# Run the async function
async def main():
    try:
        result = await unreliable_async_function()
        print(result)
    except Exception as e:
        print(e)

# To test, use asyncio to run the main function
await main()

### 5.1. Decorating Methods in Classes

A common challenge when decorating methods inside classes is handling the `self` parameter correctly.
When a decorator wraps a method, it needs to ensure the method still receives the instance (`self`) as the first argument.

Wrappers must be built for class or non class functions seperatley.

In [72]:
def method_logger(func):
    """Decorator to log method calls."""
    def wrapper(self, *args, **kwargs):  # Note the inclusion of 'self'
        print(f"Calling {func.__name__} from {self.__class__.__name__} with args: {args}, kwargs: {kwargs}")
        result = func(self, *args, **kwargs)
        print(f"{func.__name__} returned: {result}")
        return result
    return wrapper

# Class with decorated methods
class MyClass:
    def __init__(self, x):
        self.x = x

    @method_logger
    def increment(self, value):
        self.x += value
        return self.x

    @method_logger
    def decrement(self, value):
        self.x -= value
        return self.x

In [ ]:
obj = MyClass(10)
obj.increment(5)
obj.decrement(3)

### 5.2. Function Wrappers that Modify Function Signatures

Python's `inspect` module allows us to modify a function’s signature dynamically.
This can be useful when you want to create decorators that adapt to the function’s parameters and enforce certain contracts.


In [ ]:
import inspect

def add_function(a: int, b: int) -> int:
    return a + b

inspect.signature(add_function)

In [ ]:

def enforce_types(func):
    """Decorator to enforce type annotations of function parameters."""
    sig = inspect.signature(func)
    def wrapper(*args, **kwargs):
        bound_args = sig.bind(*args, **kwargs)
        for name, value in bound_args.arguments.items():
            expected_type = sig.parameters[name].annotation
            if expected_type != inspect.Parameter.empty and not isinstance(value, expected_type):
                raise TypeError(f"Argument {name} must be of type {expected_type}")
        return func(*args, **kwargs)
    return wrapper

# Function with type annotations
@enforce_types
def concatenate_strings(a: str, b: str) -> str:
    return a + b

In [ ]:
# Call the function with valid types
concatenate_strings("Hello, ", "World")

In [ ]:
# Call the function with invalid types (this will raise a TypeError)
try:
    concatenate_strings("Hello, ", 100)
except TypeError as e:
    print(e)

### 5.3. Contextual Decorators (Interacting with External Systems)

In real-world scenarios, decorators often need to interact with external systems like logging frameworks, databases, or APIs.
Here’s how you could build a decorator that logs to an external logging system or sends metrics to an external monitoring service.

In [84]:

import logging

# Set up a logging configuration
logging.basicConfig(level=logging.INFO)

def log_to_external_system(func):
    """Decorator to log function execution to an external logging system."""
    def wrapper(*args, **kwargs):
        logging.info(f"Calling {func.__name__} with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        logging.info(f"{func.__name__} returned: {result}")
        return result
    return wrapper

# Simulate an external monitoring system
def send_to_monitoring_system(func):
    """Decorator to send metrics to an external monitoring system."""
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f"Big Brother gets: Function {func.__name__} executed with result={result}")
        return result
    return wrapper

# Apply both decorators to a function
@log_to_external_system
@send_to_monitoring_system
def compute_sum(a, b):
    return a + b

In [ ]:
# Call the function to see logging and monitoring collection
compute_sum(5, 10)

### 5.4. Decorators to Modify Class Behavior

Function wrappers can be applied to **entire classes**, modifying the behavior of class instances.
For example, a decorator could be used to automatically log changes to object attributes or manage resources like a database connection.

In [87]:
def track_attribute_changes(cls):
    """Class decorator to track changes to attributes."""
    original_setattr = cls.__setattr__
    
    def new_setattr(self, name, value):
        print(f"Attribute {name} is being set to {value}")
        original_setattr(self, name, value)
    
    cls.__setattr__ = new_setattr
    return cls

# Apply the class decorator
@track_attribute_changes
class TrackedClass:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [ ]:
# Instantiate and modify object attributes
obj = TrackedClass(10, 20)
obj.x = 30  # This change will be logged
obj.y = 40  # This change will be logged as well

### 5.5. Metaprogramming with Decorators (Auto-Implementations)

Advanced decorators can even auto-implement certain methods or inject functionality into classes dynamically.
This can be useful for adding common methods (like __repr__ or equality checks) without writing them manually.

In [92]:

def auto_repr(cls):
    """Class decorator to automatically generate __repr__ based on class attributes."""
    def repr_func(self):
        class_name = self.__class__.__name__
        attributes = ', '.join(f'{k}={v!r}' for k, v in self.__dict__.items())
        return f'{class_name}({attributes})'
    
    cls.__repr__ = repr_func
    return cls

# Apply the decorator to a class
@auto_repr
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

class UndecoratedPerson:
    def __init__(self, name, age):
        self.name = name
        self.age = age

In [ ]:
p = Person("Phil", 34)
print(p)

In [ ]:
p = UndecoratedPerson("Phil", 34)
print(p)

### 5.5 Class Factory Decorators

In [101]:
class Dog:
    def __init__(self, name):
        self.name = name
    
    def speak(self):
        return f"{self.name} says Woof!"

    def walk(self):
        return f"{self.name} is walking."
    
class Cat:
    def __init__(self, name):
        self.name = name
    
    def speak(self):
        return f"{self.name} says Meow!"

def animal_factory(func):
    """Factory decorator that returns an instance of an animal class with attributes."""
    def wrapper(*args, **kwargs):
        animal_type, name = func(*args, **kwargs)  # Get animal type and name from the function
        if animal_type == 'dog':
            return Dog(name)
        elif animal_type == 'cat':
            return Cat(name)
        else:
            raise ValueError(f"Unknown animal type: {animal_type}")
    return wrapper


@animal_factory
def get_animal_with_name(animal_type, name):
    """Returns the type of animal and its name."""
    return animal_type, name

In [106]:

hugo = get_animal_with_name('dog', 'Hugo')
felix = get_animal_with_name('cat', 'Felix')

In [ ]:
hugo.speak() 

In [ ]:
felix.speak()

In [ ]:
hugo.walk()

In [ ]:
try:
    felix.walk()
except AttributeError as e:
    print(e)